# 1. Dropout in Pytorch

https://pytorch.org/docs/stable/nn.html#torch.nn.Dropout

In [1]:
import torch
import torch.nn as nn

In [2]:
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Linear(2,5),
            nn.Dropout(0.5)
        )       
        
        self.layer2 = nn.Linear(5,2)
        
    def forward(self,x):
        out = self.layer1(x)
        print(out)
        out = self.layer2(out)

        return out

In [3]:
model = NN()

In [4]:
a = torch.tensor((1., 2.))
print("a :", a)

b = model(a)
print("b :", b)

a : tensor([1., 2.])
tensor([-0.0000, 0.0058, 0.0000, -0.0000, 0.5765], grad_fn=<MulBackward0>)
b : tensor([-0.3021, -0.1262], grad_fn=<AddBackward0>)


In [5]:
model.eval()
model.training

False

In [6]:
a = torch.tensor((1., 2.))
print("a :", a)

b = model(a)
print("b :", b)

a : tensor([1., 2.])
tensor([-0.1097,  0.0029,  2.2046, -0.4009,  0.2883], grad_fn=<AddBackward0>)
b : tensor([-0.5857, -0.7403], grad_fn=<AddBackward0>)


# 2. Treat gray images for color images based model

In [7]:
import torch
import torch.nn as nn

from torchvision import models
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import torchvision.utils
import torch.utils.data as Data

import numpy as np

In [8]:
import matplotlib.pyplot as plt
%matplotlib inline

## 2-1. Concatenate gray images

In [9]:
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1))
])

train_data = dsets.MNIST(root='data/',
                          train=True,
                          transform=train_transform,
                          download=True)

train_loader = Data.DataLoader(train_data, batch_size=100, shuffle=True) # num_workers : error

In [10]:
dataiter = iter(train_loader)
images, labels = dataiter.next()
print(images[0].shape)

torch.Size([3, 28, 28])


## 2-2. Change the first conv layer

In [11]:
model = models.inception_v3(pretrained=True)

In [12]:
model

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, t

In [13]:
model.Conv2d_1a_3x3.conv = nn.Conv2d(1, 32, 3, stride = 2, bias = False)

In [14]:
model

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, t

# 3. Change requires_grad for specific layers

In [15]:
model.aux_logits = False

fine_tune = False

if not fine_tune:
    for parameter in model.parameters():
        parameter.requires_grad = False

In [16]:
for parameter in model.Conv2d_1a_3x3.conv.parameters() :
    print("Before : ", parameter.requires_grad)
    parameter.requires_grad = True
    print("After : ",parameter.requires_grad)    

Before :  False
After :  True


In [17]:
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 10),
    nn.Linear(10, 2)
)

In [18]:
for m in model.modules():
    if isinstance(m, nn.Conv2d):
        for parameter in m.parameters() :
            print(m, " : ", parameter.requires_grad)
        break

    elif isinstance(m, nn.Linear):
        for parameter in m.parameters() :
            print(parameter.requires_grad)

Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)  :  True
